In [1]:
import pickle
import re
import string
import nltk
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity
from gensim import corpora, models, similarities, matutils
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [61]:
with open('../podcasts/podcasts/jhs.pickle', 'rb') as read_jhs:
    jhs_df = pickle.load(read_jhs)

In [62]:
with open('../podcasts/podcasts/aoc.pickle', 'rb') as read_aoc:
    pdf_df = pickle.load(read_aoc)

In [63]:
df = pd.concat([jhs_df,pdf_df],ignore_index=True)

In [8]:
df.head()

,url,title,transcript,source,cleaned
0,https://www.jordanharbinger.com/should-a-cheat...,469: Should a Cheater Get a Second Chance? | F...,"[<p><b>Jordan Harbinger: </b><span style=""font...",JHS,Jordan Harbinger: Welcome to Feedback Friday. ...
1,https://www.jordanharbinger.com/going-to-north...,435: Going to North Korea: Part One | Stereo S...,[ Welcome to the show. I'm Jordan Harbinger. O...,JHS,Welcome to the show. I'm Jordan Harbinger. On...
2,https://www.jordanharbinger.com/stuart-ritchie...,436: Stuart Ritchie | The Science Fictions Und...,[ This podcast is brought to you by Microsoft ...,JHS,This podcast is brought to you by Microsoft T...
3,https://www.jordanharbinger.com/koch-and-hooks...,437: Charles Koch & Brian Hooks | Bottom-Up So...,"[ Coming up on The Jordan Harbinger Show. , [0...",JHS,Coming up on The Jordan Harbinger Show. It's...
4,https://www.jordanharbinger.com/how-to-straigh...,438: How to Straighten Out Your Crooked Boss |...,"[<p><b>Jordan Harbinger: </b><span style=""font...",JHS,Jordan Harbinger: Welcome to Feedback Friday. ...


In [17]:
alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x)
punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower())
df['transcript'] = df.transcript.map(alphanumeric).map(punc_lower)
df.head()

TypeError: expected string or bytes-like object

In [68]:
#(541, 44569) With only stop_words='english'
cv1 = CountVectorizer(stop_words='english',strip_accents='unicode',max_df=.4,min_df=10,max_features=5000)
countvect = cv1.fit_transform(df['cleaned'])
new_df = pd.DataFrame(countvect.toarray(), columns=cv1.get_feature_names(),index=df['title'])

In [10]:
with open('countvect.pickle', 'wb') as cv_pickle:
    pickle.dump(countvect, cv_pickle)

In [69]:
#no max/min dfs was 44569, .5/5 was 16640, .4/10=10568
new_df.shape

(541, 5000)

In [70]:
tfidf1 = TfidfVectorizer(stop_words='english',strip_accents='unicode',max_df=.4,min_df=10,max_features=5000)
_tfidf1 = tfidf1.fit_transform(df['cleaned'])
new_df2 = pd.DataFrame(_tfidf1.toarray(), columns=tfidf1.get_feature_names(),index=df['title'])

In [13]:
with open('tfidf.pickle', 'wb') as tfidf_pickle:
    pickle.dump(_tfidf1, tfidf_pickle)

In [24]:
new_df2.shape

(541, 44569)

In [9]:
cv2 = CountVectorizer(stop_words='english',strip_accents='unicode')
countvect2 = cv2.fit_transform(df['cleaned'])
new_df3 = pd.DataFrame(countvect2.toarray(), columns=cv2.get_feature_names(),index=df['title'])

In [17]:
df[df['cleaned'].str.contains('zzjjhjz')]

,url,title,transcript,source,cleaned
266,https://www.jordanharbinger.com/nir-eyal-contr...,250: Nir Eyal | Control Your Attention and Cho...,"[<p><b>Jordan Harbinger:</b><span style=""font-...",JHS,Welcome to the show. I'm Jordan Harbinger. As...


In [18]:
print(df.loc[266,'cleaned'])

 Welcome to the show. I'm Jordan Harbinger. As always, I'm here with producer, Jason DeFillippo. On The Jordan Harbinger Show, we decode the stories, secrets, and skills of the world's most brilliant and interesting people and turn their wisdom into practical advice that you can use to impact your own life and those around you. Chase Jarvis, he's an award-winning artist, entrepreneur ,and one of the most influential photographers of the past decade and he's a friend of mine, which always works well here on the show. He's worked for huge brands like Apple and Red Bull. However, he's also been in some big slumps, lost his creativity, found it again, sacrificed it on the altar of Silicon Valley capitalism like many of us have, and then reclaimed it. Not a bad ride. Today, we'll discuss some of the most common barriers to creativity and our pursuit of a creative outlet or career, whether that's work or family obligations or just a lack of know-how. Of course, we'll also show you how to wor

In [71]:
lsa1 = TruncatedSVD(5)
cv_topic = lsa1.fit_transform(countvect)
lsa1.explained_variance_ratio_

array([0.11253648, 0.01942992, 0.0441552 , 0.02905538, 0.02125885])

In [72]:
cv_topic_word = pd.DataFrame(lsa1.components_.round(3),
             index = ["component_1","component_2","component_3","component_4","component_5"],
             columns = cv1.get_feature_names())
cv_topic_word

,00,01,10th,120,13,14,150,16,17,18,...,youngest,youre,youth,youve,yup,zealand,zip,ziprecruiter,zone,zoom
component_1,0.015,0.002,0.001,0.001,0.008,0.008,0.004,0.005,0.006,0.010,...,0.002,0.439,0.001,0.103,0.001,0.001,0.001,0.004,0.010,0.003
component_2,0.038,0.004,0.003,0.003,0.020,0.025,0.013,0.022,0.020,0.028,...,0.004,-0.084,0.006,-0.015,0.011,0.004,0.005,0.041,0.015,0.019
component_3,0.014,0.004,0.002,0.000,0.003,0.002,0.001,0.003,0.004,0.002,...,-0.000,-0.061,0.003,-0.039,0.002,0.001,0.001,0.009,0.001,0.004
component_4,-0.010,-0.004,-0.003,-0.002,-0.009,-0.012,-0.006,-0.013,-0.010,-0.006,...,-0.002,0.044,-0.001,0.012,-0.007,-0.003,0.001,-0.030,-0.008,-0.011
component_5,0.011,0.000,-0.000,-0.001,0.017,-0.004,0.001,0.003,-0.003,-0.001,...,0.001,-0.028,-0.001,-0.002,-0.001,-0.000,-0.000,-0.006,-0.004,-0.002


In [73]:
lsa2 = TruncatedSVD(5)
tfidf_topic = lsa2.fit_transform(_tfidf1)
lsa2.explained_variance_ratio_

array([0.00516999, 0.0433951 , 0.01761116, 0.01186522, 0.00924752])

In [74]:
tfidf_topic_word = pd.DataFrame(lsa2.components_.round(3),
             index = ["component_1","component_2","component_3","component_4","component_5"],
             columns = tfidf1.get_feature_names())
tfidf_topic_word

,00,01,10th,120,13,14,150,16,17,18,...,youngest,youre,youth,youve,yup,zealand,zip,ziprecruiter,zone,zoom
component_1,0.026,0.005,0.006,0.005,0.019,0.024,0.014,0.022,0.020,0.023,...,0.007,0.086,0.009,0.021,0.015,0.007,0.007,0.058,0.019,0.032
component_2,-0.004,-0.000,-0.001,-0.001,-0.003,-0.003,-0.002,-0.004,-0.003,-0.002,...,-0.000,0.485,-0.002,0.118,-0.005,-0.001,-0.001,-0.017,0.005,-0.009
component_3,0.004,-0.003,-0.005,-0.002,-0.005,-0.004,-0.002,-0.009,-0.004,-0.002,...,-0.004,0.023,0.005,0.006,-0.001,-0.003,0.010,-0.036,-0.003,0.012
component_4,-0.010,-0.001,0.001,-0.000,0.007,0.005,0.006,0.009,-0.001,0.006,...,0.002,-0.037,0.002,-0.008,-0.007,0.004,0.002,0.017,-0.001,-0.015
component_5,-0.003,-0.002,-0.001,-0.002,0.004,-0.001,-0.004,0.002,0.001,0.003,...,0.002,-0.004,0.006,0.007,0.009,-0.002,-0.001,-0.102,-0.005,0.001


In [75]:
nmf1_model = NMF(5,max_iter=20000)
nmf1_topic = nmf1_model.fit_transform(countvect)

/Users/chuckcao/anaconda3/lib/python3.7/site-packages/sklearn/decomposition/_nmf.py:315: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  "'nndsvda' in 1.1 (renaming of 0.26)."), FutureWarning)


In [76]:
nmf1_topic_word = pd.DataFrame(nmf1_model.components_.round(3),
             index = ["component_1","component_2","component_3","component_4","component_5"],
             columns = cv1.get_feature_names())
nmf1_topic_word

,00,01,10th,120,13,14,150,16,17,18,...,youngest,youre,youth,youve,yup,zealand,zip,ziprecruiter,zone,zoom
component_1,0.202,0.024,0.007,0.017,0.121,0.156,0.068,0.041,0.090,0.194,...,0.035,13.337,0.000,3.300,0.000,0.005,0.013,0.000,0.250,0.000
component_2,1.042,0.166,0.122,0.100,0.570,0.722,0.372,0.648,0.579,0.720,...,0.113,0.000,0.153,0.000,0.311,0.118,0.105,1.256,0.466,0.562
component_3,0.234,0.090,0.044,0.000,0.044,0.000,0.000,0.000,0.044,0.000,...,0.000,4.295,0.043,0.489,0.000,0.000,0.000,0.009,0.046,0.000
component_4,0.285,0.000,0.000,0.000,0.075,0.081,0.038,0.016,0.069,0.205,...,0.000,0.000,0.054,0.000,0.008,0.000,0.077,0.000,0.035,0.035
component_5,0.235,0.010,0.001,0.000,0.346,0.004,0.051,0.095,0.000,0.052,...,0.049,2.292,0.000,0.495,0.000,0.000,0.000,0.000,0.000,0.000


In [77]:
nmf2_model = NMF(5,max_iter=2000)
nmf2_topic = nmf2_model.fit_transform(_tfidf1)

/Users/chuckcao/anaconda3/lib/python3.7/site-packages/sklearn/decomposition/_nmf.py:315: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  "'nndsvda' in 1.1 (renaming of 0.26)."), FutureWarning)


In [78]:
nmf2_topic_word = pd.DataFrame(nmf2_model.components_.round(3),
             index = ["component_1","component_2","component_3","component_4","component_5"],
             columns = tfidf1.get_feature_names())
nmf2_topic_word

,00,01,10th,120,13,14,150,16,17,18,...,youngest,youre,youth,youve,yup,zealand,zip,ziprecruiter,zone,zoom
component_1,0.070,0.013,0.017,0.014,0.046,0.057,0.032,0.054,0.052,0.055,...,0.017,0.000,0.020,0.000,0.042,0.016,0.011,0.157,0.044,0.087
component_2,0.006,0.001,0.002,0.001,0.004,0.007,0.002,0.001,0.004,0.007,...,0.003,1.061,0.000,0.258,0.000,0.000,0.001,0.000,0.019,0.000
component_3,0.021,0.000,0.000,0.000,0.004,0.006,0.005,0.000,0.005,0.010,...,0.000,0.000,0.013,0.000,0.006,0.000,0.022,0.000,0.005,0.034
component_4,0.006,0.004,0.005,0.005,0.025,0.030,0.019,0.034,0.015,0.031,...,0.011,0.000,0.006,0.000,0.007,0.013,0.007,0.060,0.017,0.006
component_5,0.000,0.000,0.004,0.001,0.011,0.000,0.005,0.002,0.001,0.000,...,0.000,0.000,0.009,0.004,0.000,0.000,0.001,0.006,0.001,0.000


In [79]:
corpus1 = matutils.Sparse2Corpus(countvect)

In [80]:
id2word1 = dict((v, k) for k, v in cv1.vocabulary_.items())

In [22]:
len(id2word1)

16640

In [81]:
lda1 = models.LdaModel(corpus=corpus1, num_topics=5, id2word=id2word1, passes=20)

2021-02-19 21:54:08,166 : INFO : using symmetric alpha at 0.2
2021-02-19 21:54:08,169 : INFO : using symmetric eta at 0.2
2021-02-19 21:54:08,171 : INFO : using serial LDA version on this node
2021-02-19 21:54:08,177 : INFO : running online (multi-pass) LDA training, 5 topics, 20 passes over the supplied corpus of 5000 documents, updating model once every 2000 documents, evaluating perplexity every 5000 documents, iterating 50x with a convergence threshold of 0.001000
2021-02-19 21:54:08,265 : INFO : PROGRESS: pass 0, at document #2000/5000
2021-02-19 21:54:09,908 : INFO : merging changes from 2000 documents into a model of 5000 documents
2021-02-19 21:54:09,911 : INFO : topic #0 (0.200): 0.010*"barely" + 0.007*"balanced" + 0.007*"balance" + 0.006*"arrest" + 0.006*"airplane" + 0.005*"apart" + 0.005*"amazon" + 0.005*"ambition" + 0.005*"approaching" + 0.005*"assuming"
2021-02-19 21:54:09,912 : INFO : topic #1 (0.200): 0.008*"birth" + 0.005*"accent" + 0.005*"blah" + 0.005*"beers" + 0.004*

2021-02-19 21:54:20,430 : INFO : topic #3 (0.200): 0.010*"assets" + 0.009*"accent" + 0.009*"application" + 0.009*"articles" + 0.008*"10th" + 0.007*"athlete" + 0.007*"asleep" + 0.007*"audio" + 0.006*"attack" + 0.006*"aunt"
2021-02-19 21:54:20,436 : INFO : topic #4 (0.200): 0.011*"blue" + 0.008*"basics" + 0.008*"bat" + 0.007*"3030" + 0.006*"asap" + 0.005*"actions" + 0.005*"bias" + 0.004*"additional" + 0.004*"35" + 0.004*"3d"
2021-02-19 21:54:20,437 : INFO : topic diff=0.044795, rho=0.471405
2021-02-19 21:54:20,611 : INFO : PROGRESS: pass 2, at document #2000/5000
2021-02-19 21:54:22,364 : INFO : merging changes from 2000 documents into a model of 5000 documents
2021-02-19 21:54:22,368 : INFO : topic #0 (0.200): 0.012*"arrest" + 0.011*"31" + 0.011*"14" + 0.011*"barely" + 0.011*"balance" + 0.010*"barriers" + 0.009*"ambition" + 0.009*"01" + 0.009*"balanced" + 0.008*"almonds"
2021-02-19 21:54:22,369 : INFO : topic #1 (0.200): 0.020*"birth" + 0.017*"blew" + 0.016*"berlin" + 0.016*"blend" + 0.

2021-02-19 21:54:31,238 : INFO : topic #3 (0.200): 0.011*"accent" + 0.011*"assets" + 0.010*"application" + 0.009*"10th" + 0.009*"articles" + 0.008*"athlete" + 0.008*"audio" + 0.008*"attack" + 0.008*"asleep" + 0.007*"2017"
2021-02-19 21:54:31,240 : INFO : topic #4 (0.200): 0.007*"basics" + 0.007*"bat" + 0.007*"3030" + 0.006*"asap" + 0.005*"actions" + 0.004*"appreciated" + 0.004*"additional" + 0.004*"26" + 0.004*"80s" + 0.004*"ad"
2021-02-19 21:54:31,241 : INFO : topic diff=0.056285, rho=0.392232
2021-02-19 21:54:31,689 : INFO : PROGRESS: pass 4, at document #2000/5000
2021-02-19 21:54:33,067 : INFO : merging changes from 2000 documents into a model of 5000 documents
2021-02-19 21:54:33,070 : INFO : topic #0 (0.200): 0.014*"arrest" + 0.014*"31" + 0.013*"14" + 0.012*"barely" + 0.012*"balance" + 0.011*"barriers" + 0.011*"autopilot" + 0.011*"ambition" + 0.010*"01" + 0.010*"barsky"
2021-02-19 21:54:33,071 : INFO : topic #1 (0.200): 0.035*"blue" + 0.029*"birth" + 0.027*"berlin" + 0.027*"blew"

2021-02-19 21:54:42,081 : INFO : topic #3 (0.200): 0.012*"accent" + 0.011*"assets" + 0.010*"application" + 0.010*"10th" + 0.009*"articles" + 0.009*"athlete" + 0.008*"attack" + 0.008*"audio" + 0.008*"beers" + 0.008*"behalf"
2021-02-19 21:54:42,083 : INFO : topic #4 (0.200): 0.007*"3030" + 0.006*"basics" + 0.006*"bat" + 0.006*"asap" + 0.004*"alive" + 0.004*"appreciated" + 0.004*"26" + 0.004*"americans" + 0.004*"80s" + 0.004*"actions"
2021-02-19 21:54:42,085 : INFO : topic diff=0.057664, rho=0.342997
2021-02-19 21:54:42,146 : INFO : PROGRESS: pass 6, at document #2000/5000
2021-02-19 21:54:43,781 : INFO : merging changes from 2000 documents into a model of 5000 documents
2021-02-19 21:54:43,784 : INFO : topic #0 (0.200): 0.015*"31" + 0.015*"arrest" + 0.014*"14" + 0.013*"balance" + 0.013*"autopilot" + 0.013*"barely" + 0.012*"barriers" + 0.011*"barsky" + 0.011*"ambition" + 0.011*"awkward"
2021-02-19 21:54:43,786 : INFO : topic #1 (0.200): 0.043*"blue" + 0.032*"birth" + 0.030*"berlin" + 0.02

2021-02-19 21:54:52,943 : INFO : topic #3 (0.200): 0.013*"accent" + 0.011*"assets" + 0.010*"application" + 0.010*"10th" + 0.010*"articles" + 0.009*"attempt" + 0.009*"athlete" + 0.009*"attack" + 0.009*"beers" + 0.009*"behalf"
2021-02-19 21:54:52,944 : INFO : topic #4 (0.200): 0.007*"3030" + 0.006*"asap" + 0.006*"basics" + 0.006*"bat" + 0.005*"alive" + 0.004*"americans" + 0.004*"basketball" + 0.004*"basement" + 0.004*"aha" + 0.004*"allowing"
2021-02-19 21:54:52,945 : INFO : topic diff=0.053271, rho=0.308607
2021-02-19 21:54:53,004 : INFO : PROGRESS: pass 8, at document #2000/5000
2021-02-19 21:54:54,278 : INFO : merging changes from 2000 documents into a model of 5000 documents
2021-02-19 21:54:54,281 : INFO : topic #0 (0.200): 0.016*"31" + 0.016*"arrest" + 0.015*"14" + 0.014*"autopilot" + 0.014*"balance" + 0.013*"barely" + 0.012*"barriers" + 0.012*"barsky" + 0.012*"ambition" + 0.012*"01"
2021-02-19 21:54:54,282 : INFO : topic #1 (0.200): 0.046*"blue" + 0.033*"birth" + 0.032*"berlin" + 0

...

2021-02-19 21:55:12,019 : INFO : topic #3 (0.200): 0.013*"accent" + 0.011*"assets" + 0.011*"application" + 0.010*"10th" + 0.010*"articles" + 0.010*"attempt" + 0.009*"beers" + 0.009*"behalf" + 0.009*"attack" + 0.009*"athlete"
2021-02-19 21:55:12,020 : INFO : topic #4 (0.200): 0.007*"3030" + 0.007*"asap" + 0.006*"basics" + 0.006*"bat" + 0.005*"basketball" + 0.005*"basement" + 0.005*"americans" + 0.005*"alive" + 0.005*"allowing" + 0.005*"aha"
2021-02-19 21:55:12,022 : INFO : topic diff=0.045788, rho=0.262613
2021-02-19 21:55:12,087 : INFO : PROGRESS: pass 12, at document #2000/5000
2021-02-19 21:55:13,494 : INFO : merging changes from 2000 documents into a model of 5000 documents
2021-02-19 21:55:13,497 : INFO : topic #0 (0.200): 0.017*"31" + 0.016*"arrest" + 0.015*"14" + 0.015*"autopilot" + 0.014*"balance" + 0.014*"barely" + 0.013*"barsky" + 0.012*"barriers" + 0.012*"ambition" + 0.012*"01"
2021-02-19 21:55:13,498 : INFO : topic #1 (0.200): 0.049*"blue" + 0.034*"birth" + 0.033*"berlin" + 

2021-02-19 21:55:21,505 : INFO : topic #3 (0.200): 0.014*"accent" + 0.011*"assets" + 0.011*"application" + 0.010*"10th" + 0.010*"articles" + 0.010*"attempt" + 0.009*"beers" + 0.009*"behalf" + 0.009*"attack" + 0.009*"athlete"
2021-02-19 21:55:21,507 : INFO : topic #4 (0.200): 0.007*"3030" + 0.007*"asap" + 0.006*"basics" + 0.006*"bat" + 0.005*"basketball" + 0.005*"basement" + 0.005*"americans" + 0.005*"alive" + 0.005*"allowing" + 0.005*"aha"
2021-02-19 21:55:21,508 : INFO : topic diff=0.041475, rho=0.246183
2021-02-19 21:55:21,565 : INFO : PROGRESS: pass 14, at document #2000/5000
2021-02-19 21:55:22,999 : INFO : merging changes from 2000 documents into a model of 5000 documents
2021-02-19 21:55:23,002 : INFO : topic #0 (0.200): 0.017*"31" + 0.017*"arrest" + 0.016*"14" + 0.016*"autopilot" + 0.014*"balance" + 0.014*"barely" + 0.013*"barsky" + 0.012*"barriers" + 0.012*"ambition" + 0.012*"01"
2021-02-19 21:55:23,003 : INFO : topic #1 (0.200): 0.050*"blue" + 0.034*"birth" + 0.033*"berlin" + 

2021-02-19 21:55:31,180 : INFO : topic #3 (0.200): 0.014*"accent" + 0.011*"assets" + 0.011*"application" + 0.010*"10th" + 0.010*"articles" + 0.010*"attempt" + 0.009*"beers" + 0.009*"behalf" + 0.009*"attack" + 0.009*"audio"
2021-02-19 21:55:31,181 : INFO : topic #4 (0.200): 0.007*"3030" + 0.007*"asap" + 0.006*"basics" + 0.006*"bat" + 0.005*"basketball" + 0.005*"basement" + 0.005*"americans" + 0.005*"alive" + 0.005*"allowing" + 0.005*"aha"
2021-02-19 21:55:31,182 : INFO : topic diff=0.037808, rho=0.232495
2021-02-19 21:55:31,249 : INFO : PROGRESS: pass 16, at document #2000/5000
2021-02-19 21:55:32,647 : INFO : merging changes from 2000 documents into a model of 5000 documents
2021-02-19 21:55:32,649 : INFO : topic #0 (0.200): 0.017*"arrest" + 0.017*"31" + 0.016*"autopilot" + 0.016*"14" + 0.015*"balance" + 0.014*"barely" + 0.013*"barsky" + 0.013*"ambition" + 0.012*"01" + 0.012*"barriers"
2021-02-19 21:55:32,650 : INFO : topic #1 (0.200): 0.050*"blue" + 0.034*"birth" + 0.033*"berlin" + 0.

2021-02-19 21:55:40,121 : INFO : topic #3 (0.200): 0.014*"accent" + 0.011*"assets" + 0.011*"application" + 0.010*"10th" + 0.010*"articles" + 0.010*"attempt" + 0.009*"beers" + 0.009*"behalf" + 0.009*"attack" + 0.009*"audio"
2021-02-19 21:55:40,122 : INFO : topic #4 (0.200): 0.007*"3030" + 0.007*"asap" + 0.006*"basics" + 0.006*"bat" + 0.006*"basketball" + 0.006*"basement" + 0.005*"americans" + 0.005*"alive" + 0.005*"allowing" + 0.005*"aha"
2021-02-19 21:55:40,123 : INFO : topic diff=0.035082, rho=0.220863
2021-02-19 21:55:40,184 : INFO : PROGRESS: pass 18, at document #2000/5000
2021-02-19 21:55:41,540 : INFO : merging changes from 2000 documents into a model of 5000 documents
2021-02-19 21:55:41,544 : INFO : topic #0 (0.200): 0.017*"arrest" + 0.017*"31" + 0.016*"autopilot" + 0.016*"14" + 0.015*"balance" + 0.014*"barely" + 0.014*"barsky" + 0.013*"ambition" + 0.012*"01" + 0.012*"awkward"
2021-02-19 21:55:41,546 : INFO : topic #1 (0.200): 0.051*"blue" + 0.034*"birth" + 0.033*"berlin" + 0.0

2021-02-19 21:55:50,140 : INFO : topic #3 (0.200): 0.014*"accent" + 0.011*"assets" + 0.011*"application" + 0.010*"10th" + 0.010*"articles" + 0.010*"attempt" + 0.009*"beers" + 0.009*"behalf" + 0.009*"attack" + 0.009*"audio"
2021-02-19 21:55:50,141 : INFO : topic #4 (0.200): 0.007*"3030" + 0.007*"asap" + 0.006*"basics" + 0.006*"bat" + 0.006*"basketball" + 0.006*"basement" + 0.005*"americans" + 0.005*"alive" + 0.005*"allowing" + 0.005*"abuse"
2021-02-19 21:55:50,143 : INFO : topic diff=0.032960, rho=0.210819


In [82]:
lda1.print_topics()

2021-02-19 21:55:50,182 : INFO : topic #0 (0.200): 0.017*"arrest" + 0.017*"31" + 0.017*"14" + 0.016*"autopilot" + 0.015*"balance" + 0.014*"barsky" + 0.013*"01" + 0.013*"ambition" + 0.013*"barely" + 0.013*"awkward"
2021-02-19 21:55:50,194 : INFO : topic #1 (0.200): 0.054*"blue" + 0.033*"berlin" + 0.032*"blew" + 0.031*"blend" + 0.031*"birth" + 0.030*"biology" + 0.029*"benefits" + 0.028*"blown" + 0.028*"bike" + 0.027*"betterhelp"
2021-02-19 21:55:50,197 : INFO : topic #2 (0.200): 0.010*"applications" + 0.010*"base" + 0.009*"accept" + 0.009*"achieve" + 0.009*"appealing" + 0.009*"80" + 0.009*"alien" + 0.008*"achieved" + 0.008*"afford" + 0.007*"approval"
2021-02-19 21:55:50,200 : INFO : topic #3 (0.200): 0.014*"accent" + 0.011*"assets" + 0.011*"application" + 0.010*"10th" + 0.010*"articles" + 0.010*"attempt" + 0.009*"beers" + 0.009*"behalf" + 0.009*"attack" + 0.009*"audio"
2021-02-19 21:55:50,204 : INFO : topic #4 (0.200): 0.007*"3030" + 0.007*"asap" + 0.006*"basics" + 0.006*"bat" + 0.006*"b

[(0,
  '0.017*"arrest" + 0.017*"31" + 0.017*"14" + 0.016*"autopilot" + 0.015*"balance" + 0.014*"barsky" + 0.013*"01" + 0.013*"ambition" + 0.013*"barely" + 0.013*"awkward"'),
 (1,
  '0.054*"blue" + 0.033*"berlin" + 0.032*"blew" + 0.031*"blend" + 0.031*"birth" + 0.030*"biology" + 0.029*"benefits" + 0.028*"blown" + 0.028*"bike" + 0.027*"betterhelp"'),
 (2,
  '0.010*"applications" + 0.010*"base" + 0.009*"accept" + 0.009*"achieve" + 0.009*"appealing" + 0.009*"80" + 0.009*"alien" + 0.008*"achieved" + 0.008*"afford" + 0.007*"approval"'),
 (3,
  '0.014*"accent" + 0.011*"assets" + 0.011*"application" + 0.010*"10th" + 0.010*"articles" + 0.010*"attempt" + 0.009*"beers" + 0.009*"behalf" + 0.009*"attack" + 0.009*"audio"'),
 (4,
  '0.007*"3030" + 0.007*"asap" + 0.006*"basics" + 0.006*"bat" + 0.006*"basketball" + 0.006*"basement" + 0.005*"americans" + 0.005*"alive" + 0.005*"allowing" + 0.005*"abuse"')]

In [ ]:
#topic space

In [26]:
corpus2 = matutils.Sparse2Corpus(_tfidf1)

In [28]:
id2word2 = dict((v, k) for k, v in tfidf1.vocabulary_.items())

In [32]:
lda2 = models.LdaModel(corpus=corpus2, num_topics=3, id2word=id2word2, passes=5)

2021-02-18 20:05:25,985 : INFO : using symmetric alpha at 0.3333333333333333
2021-02-18 20:05:25,986 : INFO : using symmetric eta at 0.3333333333333333
2021-02-18 20:05:25,995 : INFO : using serial LDA version on this node
2021-02-18 20:05:26,011 : INFO : running online (multi-pass) LDA training, 3 topics, 5 passes over the supplied corpus of 44569 documents, updating model once every 2000 documents, evaluating perplexity every 20000 documents, iterating 50x with a convergence threshold of 0.001000
2021-02-18 20:05:26,023 : INFO : PROGRESS: pass 0, at document #2000/44569
2021-02-18 20:05:26,332 : INFO : merging changes from 2000 documents into a model of 44569 documents
2021-02-18 20:05:26,341 : INFO : topic #0 (0.333): 0.002*"1921" + 0.001*"40th" + 0.001*"1st" + 0.001*"221" + 0.001*"174" + 0.001*"3om" + 0.001*"150th" + 0.001*"2000s" + 0.001*"2019" + 0.001*"15"
2021-02-18 20:05:26,342 : INFO : topic #1 (0.333): 0.001*"1921" + 0.001*"353" + 0.001*"3x" + 0.001*"1955" + 0.001*"141" + 0.0

2021-02-18 20:05:28,968 : INFO : topic #0 (0.333): 0.001*"1944" + 0.001*"01" + 0.001*"164" + 0.001*"30th" + 0.001*"34" + 0.001*"113" + 0.001*"1920s" + 0.001*"1937" + 0.001*"1740" + 0.001*"238"
2021-02-18 20:05:28,970 : INFO : topic #1 (0.333): 0.000*"141" + 0.000*"130s" + 0.000*"266" + 0.000*"1800contacts" + 0.000*"1991" + 0.000*"337" + 0.000*"2001" + 0.000*"23" + 0.000*"140" + 0.000*"231"
2021-02-18 20:05:28,972 : INFO : topic #2 (0.333): 0.000*"130s" + 0.000*"266" + 0.000*"199" + 0.000*"220" + 0.000*"135" + 0.000*"237" + 0.000*"267" + 0.000*"2001" + 0.000*"32" + 0.000*"06"
2021-02-18 20:05:28,973 : INFO : topic diff=0.009811, rho=0.316228
2021-02-18 20:05:28,994 : INFO : PROGRESS: pass 0, at document #22000/44569
2021-02-18 20:05:29,174 : INFO : merging changes from 2000 documents into a model of 44569 documents
2021-02-18 20:05:29,183 : INFO : topic #0 (0.333): 0.001*"01" + 0.001*"178" + 0.001*"1944" + 0.001*"1989" + 0.001*"29th" + 0.001*"238" + 0.001*"1967" + 0.001*"2009" + 0.001*"

2021-02-18 20:05:31,448 : INFO : PROGRESS: pass 0, at document #40000/44569
2021-02-18 20:05:31,649 : INFO : merging changes from 2000 documents into a model of 44569 documents
2021-02-18 20:05:31,658 : INFO : topic #0 (0.333): 0.001*"113" + 0.001*"2200" + 0.001*"337" + 0.001*"295" + 0.001*"1937" + 0.001*"2026" + 0.001*"2000s" + 0.001*"1991" + 0.001*"34" + 0.001*"1976"
2021-02-18 20:05:31,659 : INFO : topic #1 (0.333): 0.000*"130s" + 0.000*"266" + 0.000*"2001" + 0.000*"23" + 0.000*"1800contacts" + 0.000*"141" + 0.000*"337" + 0.000*"113" + 0.000*"1991" + 0.000*"135"
2021-02-18 20:05:31,661 : INFO : topic #2 (0.333): 0.000*"130s" + 0.000*"266" + 0.000*"199" + 0.000*"237" + 0.000*"2001" + 0.000*"135" + 0.000*"267" + 0.000*"06" + 0.000*"337" + 0.000*"220"
2021-02-18 20:05:31,663 : INFO : topic diff=0.011192, rho=0.223607
2021-02-18 20:05:31,689 : INFO : PROGRESS: pass 0, at document #42000/44569
2021-02-18 20:05:31,880 : INFO : merging changes from 2000 documents into a model of 44569 docu

2021-02-18 20:05:33,714 : INFO : topic diff=0.004340, rho=0.202925
2021-02-18 20:05:33,734 : INFO : PROGRESS: pass 1, at document #14000/44569
2021-02-18 20:05:33,913 : INFO : merging changes from 2000 documents into a model of 44569 documents
2021-02-18 20:05:33,920 : INFO : topic #0 (0.333): 0.001*"113" + 0.001*"25" + 0.001*"237" + 0.001*"30th" + 0.001*"29th" + 0.001*"1972" + 0.001*"2nd" + 0.001*"223" + 0.001*"1985" + 0.001*"337"
2021-02-18 20:05:33,921 : INFO : topic #1 (0.333): 0.000*"130s" + 0.000*"266" + 0.000*"2001" + 0.000*"237" + 0.000*"2x" + 0.000*"2014" + 0.000*"23" + 0.000*"3600" + 0.000*"337" + 0.000*"1925"
2021-02-18 20:05:33,923 : INFO : topic #2 (0.333): 0.000*"130s" + 0.000*"266" + 0.000*"2001" + 0.000*"237" + 0.000*"2x" + 0.000*"06" + 0.000*"199" + 0.000*"2014" + 0.000*"3600" + 0.000*"220"
2021-02-18 20:05:33,924 : INFO : topic diff=0.003698, rho=0.202925
2021-02-18 20:05:33,945 : INFO : PROGRESS: pass 1, at document #16000/44569
2021-02-18 20:05:34,130 : INFO : mergi

2021-02-18 20:05:36,265 : INFO : topic #2 (0.333): 0.000*"130s" + 0.000*"266" + 0.000*"2014" + 0.000*"113" + 0.000*"1988" + 0.000*"2001" + 0.000*"135" + 0.000*"237" + 0.000*"06" + 0.000*"315"
2021-02-18 20:05:36,268 : INFO : topic diff=0.004669, rho=0.202925
2021-02-18 20:05:36,289 : INFO : PROGRESS: pass 1, at document #34000/44569
2021-02-18 20:05:36,464 : INFO : merging changes from 2000 documents into a model of 44569 documents
2021-02-18 20:05:36,472 : INFO : topic #0 (0.333): 0.001*"113" + 0.001*"1974" + 0.001*"1967" + 0.001*"141" + 0.001*"135" + 0.001*"337" + 0.001*"29th" + 0.001*"30" + 0.001*"2200" + 0.001*"1944"
2021-02-18 20:05:36,473 : INFO : topic #1 (0.333): 0.000*"130s" + 0.000*"266" + 0.000*"2001" + 0.000*"237" + 0.000*"135" + 0.000*"113" + 0.000*"00000" + 0.000*"23" + 0.000*"35" + 0.000*"1640"
2021-02-18 20:05:36,476 : INFO : topic #2 (0.333): 0.000*"130s" + 0.000*"266" + 0.000*"2001" + 0.000*"237" + 0.000*"135" + 0.000*"00000" + 0.000*"113" + 0.000*"35" + 0.000*"23" + 

...

2021-02-18 20:05:44,939 : INFO : merging changes from 569 documents into a model of 44569 documents
2021-02-18 20:05:44,952 : INFO : topic #0 (0.333): 0.001*"2024" + 0.001*"2009" + 0.001*"141" + 0.001*"2029" + 0.001*"113" + 0.001*"2036" + 0.001*"2a" + 0.001*"300" + 0.001*"214" + 0.001*"1922"
2021-02-18 20:05:44,954 : INFO : topic #1 (0.333): 0.000*"200s" + 0.000*"237" + 0.000*"203" + 0.000*"399" + 0.000*"266" + 0.000*"2014" + 0.000*"280" + 0.000*"380" + 0.000*"2004" + 0.000*"4000"
2021-02-18 20:05:44,955 : INFO : topic #2 (0.333): 0.000*"200s" + 0.000*"237" + 0.000*"203" + 0.000*"399" + 0.000*"266" + 0.000*"2014" + 0.000*"280" + 0.000*"380" + 0.000*"2004" + 0.000*"4000"
2021-02-18 20:05:44,956 : INFO : topic diff=0.023632, rho=0.198872
2021-02-18 20:05:44,973 : INFO : PROGRESS: pass 3, at document #2000/44569
2021-02-18 20:05:45,151 : INFO : merging changes from 2000 documents into a model of 44569 documents
2021-02-18 20:05:45,159 : INFO : topic #0 (0.333): 0.001*"2024" + 0.001*"141" 

2021-02-18 20:05:47,571 : INFO : PROGRESS: pass 3, at document #20000/44569
2021-02-18 20:05:47,778 : INFO : merging changes from 2000 documents into a model of 44569 documents
2021-02-18 20:05:47,785 : INFO : topic #0 (0.333): 0.001*"1944" + 0.001*"113" + 0.001*"30th" + 0.001*"01" + 0.001*"164" + 0.001*"1937" + 0.001*"34" + 0.001*"2009" + 0.001*"238" + 0.001*"2019"
2021-02-18 20:05:47,786 : INFO : topic #1 (0.333): 0.000*"2001" + 0.000*"130s" + 0.000*"2014" + 0.000*"237" + 0.000*"266" + 0.000*"278" + 0.000*"2060" + 0.000*"14ers" + 0.000*"113" + 0.000*"2200"
2021-02-18 20:05:47,788 : INFO : topic #2 (0.333): 0.000*"2001" + 0.000*"130s" + 0.000*"2014" + 0.000*"237" + 0.000*"266" + 0.000*"278" + 0.000*"2060" + 0.000*"14ers" + 0.000*"113" + 0.000*"2200"
2021-02-18 20:05:47,789 : INFO : topic diff=0.004024, rho=0.195052
2021-02-18 20:05:47,810 : INFO : PROGRESS: pass 3, at document #22000/44569
2021-02-18 20:05:48,027 : INFO : merging changes from 2000 documents into a model of 44569 docum

2021-02-18 20:05:50,142 : INFO : topic diff=0.002650, rho=0.195052
2021-02-18 20:05:50,627 : INFO : -8.579 per-word bound, 382.5 perplexity estimate based on a held-out corpus of 2000 documents with 686 words
2021-02-18 20:05:50,628 : INFO : PROGRESS: pass 3, at document #40000/44569
2021-02-18 20:05:50,810 : INFO : merging changes from 2000 documents into a model of 44569 documents
2021-02-18 20:05:50,817 : INFO : topic #0 (0.333): 0.001*"113" + 0.001*"2200" + 0.001*"337" + 0.001*"1937" + 0.001*"295" + 0.001*"2000s" + 0.001*"1991" + 0.001*"2026" + 0.001*"34" + 0.001*"208"
2021-02-18 20:05:50,818 : INFO : topic #1 (0.333): 0.000*"130s" + 0.000*"2001" + 0.000*"266" + 0.000*"414" + 0.000*"23" + 0.000*"237" + 0.000*"2014" + 0.000*"2007" + 0.000*"135" + 0.000*"113"
2021-02-18 20:05:50,820 : INFO : topic #2 (0.333): 0.000*"130s" + 0.000*"2001" + 0.000*"266" + 0.000*"414" + 0.000*"23" + 0.000*"237" + 0.000*"2014" + 0.000*"2007" + 0.000*"135" + 0.000*"113"
2021-02-18 20:05:50,821 : INFO : top

2021-02-18 20:05:52,685 : INFO : topic #2 (0.333): 0.000*"237" + 0.000*"130s" + 0.000*"266" + 0.000*"2001" + 0.000*"06" + 0.000*"2014" + 0.000*"23" + 0.000*"2x" + 0.000*"220" + 0.000*"31"
2021-02-18 20:05:52,686 : INFO : topic diff=0.004175, rho=0.191444
2021-02-18 20:05:52,705 : INFO : PROGRESS: pass 4, at document #14000/44569
2021-02-18 20:05:52,876 : INFO : merging changes from 2000 documents into a model of 44569 documents
2021-02-18 20:05:52,883 : INFO : topic #0 (0.333): 0.001*"113" + 0.001*"25" + 0.001*"237" + 0.001*"30th" + 0.001*"29th" + 0.001*"1972" + 0.001*"2nd" + 0.001*"223" + 0.001*"1985" + 0.001*"337"
2021-02-18 20:05:52,884 : INFO : topic #1 (0.333): 0.000*"130s" + 0.000*"2001" + 0.000*"237" + 0.000*"2x" + 0.000*"266" + 0.000*"2014" + 0.000*"383" + 0.000*"3600" + 0.000*"06" + 0.000*"31"
2021-02-18 20:05:52,885 : INFO : topic #2 (0.333): 0.000*"130s" + 0.000*"2001" + 0.000*"237" + 0.000*"2x" + 0.000*"266" + 0.000*"2014" + 0.000*"383" + 0.000*"3600" + 0.000*"06" + 0.000*"

2021-02-18 20:05:55,257 : INFO : topic #1 (0.333): 0.000*"130s" + 0.000*"266" + 0.000*"2014" + 0.000*"1988" + 0.000*"113" + 0.000*"2001" + 0.000*"135" + 0.000*"315" + 0.000*"237" + 0.000*"06"
2021-02-18 20:05:55,259 : INFO : topic #2 (0.333): 0.000*"130s" + 0.000*"266" + 0.000*"2014" + 0.000*"1988" + 0.000*"113" + 0.000*"2001" + 0.000*"135" + 0.000*"315" + 0.000*"237" + 0.000*"06"
2021-02-18 20:05:55,260 : INFO : topic diff=0.004434, rho=0.191444
2021-02-18 20:05:55,281 : INFO : PROGRESS: pass 4, at document #34000/44569
2021-02-18 20:05:55,454 : INFO : merging changes from 2000 documents into a model of 44569 documents
2021-02-18 20:05:55,462 : INFO : topic #0 (0.333): 0.001*"113" + 0.001*"1974" + 0.001*"1967" + 0.001*"141" + 0.001*"337" + 0.001*"135" + 0.001*"29th" + 0.001*"30" + 0.001*"1944" + 0.001*"2200"
2021-02-18 20:05:55,463 : INFO : topic #1 (0.333): 0.000*"130s" + 0.000*"2001" + 0.000*"266" + 0.000*"237" + 0.000*"135" + 0.000*"00000" + 0.000*"113" + 0.000*"35" + 0.000*"23" + 

In [34]:
lda2.print_topics()

2021-02-18 20:05:57,444 : INFO : topic #0 (0.333): 0.001*"2024" + 0.001*"2009" + 0.001*"113" + 0.001*"141" + 0.001*"2029" + 0.001*"2036" + 0.001*"2a" + 0.001*"300" + 0.001*"1944" + 0.001*"1922"
2021-02-18 20:05:57,447 : INFO : topic #1 (0.333): 0.000*"200s" + 0.000*"237" + 0.000*"203" + 0.000*"399" + 0.000*"266" + 0.000*"2014" + 0.000*"280" + 0.000*"380" + 0.000*"4000" + 0.000*"2004"
2021-02-18 20:05:57,449 : INFO : topic #2 (0.333): 0.000*"200s" + 0.000*"237" + 0.000*"203" + 0.000*"399" + 0.000*"266" + 0.000*"2014" + 0.000*"280" + 0.000*"380" + 0.000*"4000" + 0.000*"2004"


[(0,
  '0.001*"2024" + 0.001*"2009" + 0.001*"113" + 0.001*"141" + 0.001*"2029" + 0.001*"2036" + 0.001*"2a" + 0.001*"300" + 0.001*"1944" + 0.001*"1922"'),
 (1,
  '0.000*"200s" + 0.000*"237" + 0.000*"203" + 0.000*"399" + 0.000*"266" + 0.000*"2014" + 0.000*"280" + 0.000*"380" + 0.000*"4000" + 0.000*"2004"'),
 (2,
  '0.000*"200s" + 0.000*"237" + 0.000*"203" + 0.000*"399" + 0.000*"266" + 0.000*"2014" + 0.000*"280" + 0.000*"380" + 0.000*"4000" + 0.000*"2004"')]

In [ ]:
#topic space